In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import classificacao

sns.set_style('darkgrid')
%matplotlib inline

In [3]:
df = pd.read_csv('conhecimento.csv')
df['dia'] = df.id_cena.str[-3:].astype(int)

id_cenas = [
    # 'LS82260702016078', 
    'LS82260702017256', # 226_070
    # 'LS82210702016059', 
    'LS82210702017269', # 221_070
    # 'LS52210672002068', 
    'LS82210672017285', # 221_067
    # 'LS82210742016059', 
    'LS82210742018128', # 221_074
]
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena', 'caso']

In [4]:
def gera_teste_modelo_caso1(id_cena):
    teste_original = df[df.id_cena== id_cena].copy().reset_index(drop=True)
    teste = teste_original.copy()
    teste['verifica'] = 0

    # Modelo
    modelo = df[~(df.id_cena==id_cena)].copy()
    return teste, teste_original, modelo

def gera_teste_modelo_caso2(id_cena):
    
    teste_original = df[df.id_cena==id_cena].copy().reset_index(drop=True)
    teste = teste_original.copy()
    teste['verifica'] = 0

    # Modelo
    modelo_inteiro = df[~(df.id_cena==id_cena)].copy()

    modelo_inteiro_1 = modelo_inteiro[modelo_inteiro.verifica==1].copy()
    modelo_inteiro_3_full = modelo_inteiro[modelo_inteiro.verifica==3].copy()

    id3 = np.random.choice(modelo_inteiro_3_full.id, len(modelo_inteiro_1), replace=False) 
    modelo_inteiro_3 = modelo_inteiro_3_full[modelo_inteiro_3_full.id.isin(id3)]

    modelo = pd.concat([modelo_inteiro_1, modelo_inteiro_3]).reset_index(drop=True)

    return teste, teste_original, modelo

def gera_teste_modelo_caso3(id_cena):
    print(id_cena)
    
    dia = int(id_cena[-3:])
    print(df.shape, dia, id_cena)

    teste_original = df[df.id_cena== id_cena].copy().reset_index(drop=True)
    teste = teste_original.copy()
    teste['verifica'] = 0
    print(len(teste_original))

    # Modelo
    op = teste.orb_pto.loc[0]
    print(op)
    modelo = df[(df.dia.isin(range(dia-90, dia+90))) & (df.orb_pto == op)].copy()

    print(len(modelo))
    print(modelo.verifica.value_counts())

    return teste, teste_original, modelo

In [5]:
def get_id_cenas(df, orb_pto):
    return sorted(list(df[df.orb_pto==orb_pto].id_cena.unique()))
def resultado(teste, teste_original, verifica=1):
    r = pd.merge(teste[['id', 'classificacao']], teste_original[['id', 'verifica']], on='id')
    r['v'] = r.classificacao.astype(str) + r.verifica.astype(str)
    
    try:
        x = len(r[(r.verifica==verifica) & (r.classificacao == verifica)])
        v = len(r[r.verifica==verifica])
        p = int((x*100)/v)
    except:
        return [0, 0, 0]
    return [x, v, p]

In [ ]:
sumario = pd.DataFrame([], columns=col_sum)
for id_cena in id_cenas:
    teste, teste_original, modelo = gera_teste_modelo_caso1(id_cena)
    y_pred = classificacao.classifica(modelo, teste)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = resultado(y_pred, y_true)
    r.append(id_cena)
    r.append(1)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)
    
    teste, teste_original, modelo = gera_teste_modelo_caso2(id_cena)
    y_pred = classificacao.classifica(modelo, teste)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = resultado(y_pred, y_true)
    r.append(id_cena)
    r.append(2)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)
    
    teste, teste_original, modelo = gera_teste_modelo_caso3(id_cena)
    y_pred = classificacao.classifica(modelo, teste)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = resultado(y_pred, y_true)
    r.append(id_cena)
    r.append(3)
    sumario = sumario.append(dict(zip(col_sum, r)), ignore_index=True)

    print(r)

In [7]:
sumario

,previsao,verdade,porcentagem,id_cena,caso
0,743,805,92,LS82260702017256,1
1,795,805,98,LS82260702017256,2
2,560,805,69,LS82260702017256,3
3,281,599,46,LS82210702017269,1
4,438,599,73,LS82210702017269,2
5,67,599,11,LS82210702017269,3
6,398,1102,36,LS82210672017285,1
7,699,1102,63,LS82210672017285,2
8,182,1102,16,LS82210672017285,3
9,255,1072,23,LS82210742018128,1


In [8]:
sumario.to_csv('resultado_melhores_colunas.csv', index=False)